<a href="https://colab.research.google.com/github/SammyAlbataineh/Sports-Probability-Website/blob/main/NFL_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub transformers timm -q

import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import kagglehub

In [ ]:
import kagglehub

path = kagglehub.dataset_download("tobycrabtree/nfl-scores-and-betting-data")
print("Downloaded path:", path)

Using Colab cache for faster access to the 'nfl-scores-and-betting-data' dataset.
Downloaded path: /kaggle/input/nfl-scores-and-betting-data


In [ ]:
import pandas as pd

# Construct the full path to the desired CSV file
scores_csv_path = os.path.join(path, 'spreadspoke_scores.csv')

# Load the CSV file into a pandas DataFrame
df_scores = pd.read_csv(scores_csv_path)

# Display the first 5 rows of the DataFrame
display(df_scores.head())

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN


In [ ]:
df_scores = df_scores.drop(columns=['weather_detail'])
display(df_scores.head())

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0


In [ ]:
print(path)
for root, dirs, files in os.walk(path):
    for name in files:
        print(os.path.join(root, name))

/kaggle/input/nfl-scores-and-betting-data
/kaggle/input/nfl-scores-and-betting-data/nfl_teams.csv
/kaggle/input/nfl-scores-and-betting-data/spreadspoke_scores.csv
/kaggle/input/nfl-scores-and-betting-data/spreadspoke.R


In [ ]:
df_scores['home_win'] = (df_scores['score_home'] > df_scores['score_away']).astype(int)
y = df_scores[['score_home', 'score_away', 'home_win']]
x = df_scores.drop(columns=['score_home', 'score_away', 'home_win'])
display(df_scores.head())

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,home_win
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,0
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,1
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,1
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,0
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,1


In [ ]:
encoder = OneHotEncoder(sparse_output=False)
x_encoded = encoder.fit_transform(x[x.columns])

In [ ]:
print(df_scores.isnull().sum())

schedule_date             0
schedule_season           0
schedule_week             0
schedule_playoff          0
team_home                 0
score_home               32
score_away               32
team_away                 0
team_favorite_id       2511
spread_favorite        2511
over_under_line        2521
stadium                   0
stadium_neutral           0
weather_temperature    1544
weather_wind_mph       1561
weather_humidity       5685
home_win                  0
dtype: int64


In [ ]:
median_score_home = df_scores['score_home'].median()
median_score_away = df_scores['score_away'].median()

df_scores['score_home'].fillna(median_score_home, inplace=True)
df_scores['score_away'].fillna(median_score_away, inplace=True)

print(f"Median score home: {median_score_home}")
print(f"Median score away: {median_score_away}")
print("Missing values after filling:")
print(df_scores[['score_home', 'score_away']].isnull().sum())

Median score home: 22.0
Median score away: 20.0
Missing values after filling:
score_home    0
score_away    0
dtype: int64


/tmp/ipython-input-1885343773.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_scores['score_home'].fillna(median_score_home, inplace=True)
/tmp/ipython-input-1885343773.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

In [ ]:
median_score_home = df_scores['score_home'].median()
median_score_away = df_scores['score_away'].median()

df_scores['score_home'] = df_scores['score_home'].fillna(median_score_home)
df_scores['score_away'] = df_scores['score_away'].fillna(median_score_away)

print(f"Median score home: {median_score_home}")
print(f"Median score away: {median_score_away}")
print("Missing values after filling:")
print(df_scores[['score_home', 'score_away']].isnull().sum())

Median score home: 22.0
Median score away: 20.0
Missing values after filling:
score_home    0
score_away    0
dtype: int64


In [ ]:
print("Missing values in 'score_home' and 'score_away' after imputation:")
print(df_scores[['score_home', 'score_away']].isnull().sum())

print("\nMissing values in the entire df_scores DataFrame:")
print(df_scores.isnull().sum())

Missing values in 'score_home' and 'score_away' after imputation:
score_home    0
score_away    0
dtype: int64

Missing values in the entire df_scores DataFrame:
schedule_date             0
schedule_season           0
schedule_week             0
schedule_playoff          0
team_home                 0
score_home                0
score_away                0
team_away                 0
team_favorite_id       2511
spread_favorite        2511
over_under_line        2521
stadium                   0
stadium_neutral           0
weather_temperature    1544
weather_wind_mph       1561
weather_humidity       5685
home_win                  0
dtype: int64


In [ ]:
display(df_scores.head())

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,home_win
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,0
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,1
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,1
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,0
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,1


In [ ]:
y_target = y['home_win']
X = x_encoded

print("Shape of y_target:", y_target.shape)
print("Shape of X:", X.shape)

Shape of y_target: (14358,)
Shape of X: (14358, 3421)


In [ ]:
latest_season = df_scores['schedule_season'].max()
weights = (df_scores['schedule_season'] - df_scores['schedule_season'].min()) / \
          (latest_season - df_scores['schedule_season'].min()) + 0.1 # Add a small constant to ensure non-zero weights

print("First 5 calculated weights:")
print(weights.head())
print("Last 5 calculated weights:")
print(weights.tail())
print("Shape of weights:", weights.shape)

First 5 calculated weights:
0    0.1
1    0.1
2    0.1
3    0.1
4    0.1
Name: schedule_season, dtype: float64
Last 5 calculated weights:
14353    1.1
14354    1.1
14355    1.1
14356    1.1
14357    1.1
Name: schedule_season, dtype: float64
Shape of weights: (14358,)


In [ ]:
cutoff_season = 2023

train_mask = df_scores['schedule_season'] <= cutoff_season
test_mask = df_scores['schedule_season'] > cutoff_season

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y_target[train_mask], y_target[test_mask]
weights_train, weights_test = weights[train_mask], weights[test_mask]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Max season in training set: {df_scores[train_mask]['schedule_season'].max()}")
print(f"Min season in test set: {df_scores[test_mask]['schedule_season'].min()}")

Training set size: 13801 samples
Test set size: 557 samples
Max season in training set: 2023
Min season in test set: 2024


In [ ]:
cutoff_season = 2023

train_mask = df_scores['schedule_season'] <= cutoff_season
test_mask = df_scores['schedule_season'] > cutoff_season

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y_target[train_mask], y_target[test_mask]
weights_train, weights_test = weights[train_mask], weights[test_mask]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Max season in training set: {df_scores[train_mask]['schedule_season'].max()}")
print(f"Min season in test set: {df_scores[test_mask]['schedule_season'].min()}")

Training set size: 13801 samples
Test set size: 557 samples
Max season in training set: 2023
Min season in test set: 2024


In [ ]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of weights_train: {weights_train.shape}")
print(f"Shape of weights_test: {weights_test.shape}")

Shape of X_train: (13801, 3421)
Shape of X_test: (557, 3421)
Shape of y_train: (13801,)
Shape of y_test: (557,)
Shape of weights_train: (13801,)
Shape of weights_test: (557,)


In [ ]:
model = LogisticRegression(solver='liblinear', random_state=42, max_iter=2000)
model.fit(X_train, y_train, sample_weight=weights_train)

print("Logistic Regression model successfully fitted to the training data with time-decaying weights.")

Logistic Regression model successfully fitted to the training data with time-decaying weights.


In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy on Test Set: {accuracy:.4f}")

Model Accuracy on Test Set: 0.5530
